In [1]:
# %pip install langchain_mistralai

In [2]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

from langchain.memory import ConversationBufferMemory

from langchain.chat_models import ChatOpenAI
from langchain.agents.format_scratchpad import format_to_openai_function_messages

from langchain.agents import (
    initialize_agent,
    OpenAIFunctionsAgent,
    AgentExecutor,
    AgentType,
)

from gdoc_sync.integrations.projects.tools import (
    run_sqlite_query,
    list_tables,
    describe_tables,
)

In [3]:
base_url = "http://172.21.240.1:1234/v1"

chat = ChatOpenAI(
    base_url=base_url, temperature=0, openai_api_key="not_required", verbose=True
)

In [4]:
tools = [run_sqlite_query, describe_tables]

In [5]:
def sqlite_agent(query: str, llm) -> str:
    PREFIX = f"""You are a helpful AI with access to a Sqlite database with tables: \n{list_tables()}"""
    agent = initialize_agent(
        tools=tools,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        llm=llm,
        agent_kwargs={"prefix": PREFIX},
    )

    return agent.run(query)

In [6]:
sqlite_agent("how many users have shipping addresses?", llm=chat)



> Entering new AgentExecutor chain...
To find out how many users have a shipping address, I need to join the users and addresses tables on their common id field. Then I can use the count function to get the number of rows in the result set.

Action: run_sqlite_query
Action Input: SELECT COUNT(*) FROM users JOIN addresses ON users.id = addresses.user_id;
Observation: [(4012,)]
Thought:

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `The final answer to the original input question is that there are 4012 users with shipping addresses.`